In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import torch
import dgl
import os
from tqdm import tqdm
import jieba
import jieba.analyse
import numpy as np

from parallel_processor import process_data
from functools import partial

from itertools import chain

from collections import Counter

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
with open('data/thucnews_text_only.txt') as fin:
    text = [item.replace('\n', '') for item in fin.readlines()]

In [3]:
text = chain.from_iterable([item.split('。') for item in tqdm(text)])
text = list(text)

100%|██████████| 174132/174132 [00:00<00:00, 321782.33it/s]


In [4]:
label_names = ['娱乐','彩票','教育','时政','游戏','科技','财经','体育','家居','房产','时尚','星座','社会','股票']
label_names_set = set(label_names)

In [5]:
allow_pos = ('an', 'n', 'nr', 'ns', 'nt')

In [6]:
jieba.analyse.extract_tags('我了个擦')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.654 seconds.
Prefix dict has been built successfully.


[]

In [57]:
def extract_tags(docs, top_k=10, allow_pos=None):
    all_doc_tags = []
    for doc in tqdm(docs):
        tags = jieba.analyse.extract_tags(doc, topK=top_k, withWeight=False, allowPOS=allow_pos)
        
        if len(label_names_set.intersection(tags)) > 0:
            all_doc_tags.append(tags)
    return all_doc_tags


extract_noun = partial(extract_tags, top_k=100, allow_pos=allow_pos)


In [58]:
result = process_data(np.array(text), extract_noun, num_workers=22)

100%|██████████| 68816/68816 [02:40<00:00, 428.78it/s]
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [59]:
result = result.tolist()
len(result)

66410

In [40]:
# sport_docs = [item for item in result if '体育' in item]
# sport_docs[:10]

In [60]:
def freq_filter(data, min_freq=1):
    """
    过滤低频词
    """
    cnter = dict(Counter(list(chain.from_iterable(data))))
    cnter = {k: cnter[k] for k in sorted(cnter, key=lambda x: cnter[x], reverse=True) if cnter[k] > min_freq}
    return set(cnter.keys())

In [99]:
result = [[jtem for jtem in item if jtem not in inter_words]  for item in tqdm(result)]

100%|██████████| 66410/66410 [00:31<00:00, 2118.20it/s]


In [100]:
all_words = freq_filter(result, min_freq=3)
len(all_words)

8339

In [101]:
w2i = {w: i for i, w in enumerate(all_words)}
i2w = {v: k for k, v in w2i.items()}

In [102]:
g_mat = np.zeros([len(all_words), len(all_words)])
g_mat.shape

(8339, 8339)

In [103]:
print(f'graph size in mem: {g_mat.size / 1024 / 1024 / 1024}')

graph size in mem: 0.06476316694170237


In [104]:
for doc in tqdm(result):
    doc = set(doc)
    for u in doc:
        for v in doc:
            if not w2i.get(u) or not w2i.get(v): continue
            if u == v: continue
            g_mat[w2i[u], w2i[v]] += 1

100%|██████████| 66410/66410 [00:00<00:00, 86936.19it/s]


In [105]:
g = nx.from_numpy_array(g_mat)
g

In [106]:
g = dgl.DGLGraph(g)
g

Graph(num_nodes=8339, num_edges=163794,
      ndata_schemes={}
      edata_schemes={})

In [107]:
N = len(all_words)
DAMP = 0.85

In [108]:
import dgl.function as fn

def pagerank(g):
    g.ndata['pv'] = g.ndata['pv'] / g.ndata['deg']
    
    g.update_all(message_func=fn.copy_src(src='pv', out='m'), 
                 reduce_func=fn.sum(msg='m', out='m_sum'))
    
    g.ndata['pv'] = (1 - DAMP) / N + DAMP * g.ndata['m_sum']

In [109]:
g.ndata['pv'] = torch.ones(N) / N
g.ndata['deg'] = g.out_degrees(g.nodes()).float()
pagerank(g)

In [110]:
g.ndata['pv']

tensor([1.7988e-05, 2.6641e-04, 9.1100e-05,  ..., 4.7911e-05, 3.2744e-05,
        1.3186e-04])

In [111]:
# 用1-hop的pv作为排序
cate_words = {}
for ln in label_names:
    cate_words.setdefault(ln, [])
    try:
        adjoin_nodes = [idx for idx, val in enumerate(g_mat[w2i[ln]]) if val != 0]
        print(f'"{ln}" has {len(adjoin_nodes)} adjoin nodes.')

        top_idxs = torch.topk(g.ndata['pv'][adjoin_nodes], 100)[1]
#         top_idxs = torch.topk(g.ndata['pv'][adjoin_nodes], np.sum(np.array(adjoin_nodes) > 0))[1]

        words = []
        for idx in top_idxs:
            words.append(i2w[adjoin_nodes[idx.item()]])

        cate_words[ln].extend(words)
    except:
        print(ln, 'has no adjoins.')

娱乐 has no adjoins.
"彩票" has 1571 adjoin nodes.
教育 has no adjoins.
"时政" has 24 adjoin nodes.
时政 has no adjoins.
"游戏" has 4236 adjoin nodes.
"科技" has 3004 adjoin nodes.
"财经" has 616 adjoin nodes.
体育 has no adjoins.
"家居" has 1427 adjoin nodes.
房产 has no adjoins.
"时尚" has 2249 adjoin nodes.
"星座" has 697 adjoin nodes.
"社会" has 3479 adjoin nodes.
"股票" has 1253 adjoin nodes.


In [112]:
cate_words

{'娱乐': [],
 '彩票': ['游戏',
  '社会',
  '科技',
  '时尚',
  '股票',
  '财经',
  '星座',
  '投注站',
  '单式',
  '中奖号码',
  '中得',
  '公益金',
  '奖池',
  '司法',
  '奖金额',
  '蓝球',
  '销售点',
  '复式票',
  '单人',
  '竞彩',
  '教育部',
  '足球彩票',
  '买彩',
  '武功',
  '彩站',
  '机选',
  '红球',
  '氏族',
  '排列五',
  '开奖号码',
  '河南省',
  '公安部',
  '私彩',
  '客场',
  '英里',
  '纲要',
  '出票',
  '民政部',
  '医学',
  '事业单位',
  '民政',
  '小伙子',
  '赔率',
  '西甲',
  '社会科学',
  '投注额',
  '进球',
  '作弊',
  '中出',
  '半球',
  '治安',
  '通报',
  '马甲',
  '初盘',
  '店主',
  '开发人员',
  '社会福利',
  '奖号',
  '单张',
  '政策措施',
  '零钱',
  '本赛季',
  '图标',
  '横幅',
  '欧冠',
  '光盘',
  '守号',
  '平手',
  '管理体制',
  '中奖人',
  '林先生',
  '蜘蛛',
  '最底层',
  '赛程',
  '大富翁',
  '型彩票',
  '铁杆',
  '西路',
  '一键',
  '武汉大学',
  '女娲',
  '平局',
  '巴萨',
  '法学院',
  '低收入',
  '行政部门',
  '冠亚军',
  '公款',
  '严肃查处',
  '英豪',
  '党和政府',
  '店员',
  '擂台赛',
  '窗帘',
  '财务管理',
  '攻城战',
  '社会保险',
  '门票',
  '趣味性',
  '私信'],
 '教育': [],
 '时政': [],
 '游戏': ['社会',
  '科技',
  '时尚',
  '彩票',
  '家居',
  '股票',
  '财经',
  '星座',
  '投注站',
  '资料片',
  '单机游戏',
  '机身',


In [79]:
intersection = set()
cnt = Counter(list(chain.from_iterable(list(cate_words.values()))))
cnt = {k: cnt[k] for k in sorted(cnt, key=lambda x: cnt[x], reverse=True)}
print(len(cnt))
cnt

9934


{'中国': 10,
 '时间': 10,
 '北京': 10,
 '市场': 10,
 '问题': 10,
 '记者': 10,
 '方面': 10,
 '经济': 10,
 '大家': 10,
 '企业': 10,
 '美国': 10,
 '国家': 10,
 '行业': 10,
 '中心': 10,
 '内容': 10,
 '情况': 10,
 '手机': 10,
 '全国': 10,
 '媒体': 10,
 '技术': 10,
 '部分': 10,
 '全球': 10,
 '信息': 10,
 '任务': 10,
 '历史': 10,
 '朋友': 10,
 '专业': 10,
 '领域': 10,
 '英国': 10,
 '城市': 10,
 '上海': 10,
 '电影': 10,
 '感觉': 10,
 '项目': 10,
 '故事': 10,
 '主题': 10,
 '集团': 10,
 '原因': 10,
 '基本': 10,
 '政府': 10,
 '背景': 10,
 '专家': 10,
 '大学': 10,
 '地方': 10,
 '学生': 10,
 '旗下': 10,
 '网站': 10,
 '论坛': 10,
 '重点': 10,
 '团队': 10,
 '人士': 10,
 '精神': 10,
 '规定': 10,
 '资源': 10,
 '单位': 10,
 '东西': 10,
 '新闻': 10,
 '经历': 10,
 '阶段': 10,
 '商业': 10,
 '地点': 10,
 '事业': 10,
 '深圳': 10,
 '数字': 10,
 '工程': 10,
 '高端': 10,
 '交流': 10,
 '规划': 10,
 '区域': 10,
 '品质': 10,
 '主持人': 10,
 '话题': 10,
 '热情': 10,
 '办法': 10,
 '联系': 10,
 '意识': 10,
 '评价': 10,
 '规范': 10,
 '顶级': 10,
 '电视': 10,
 '影响力': 10,
 '文章': 10,
 '总裁': 10,
 '亚洲': 10,
 '总体': 10,
 '双方': 10,
 '公众': 10,
 '观点': 10,
 '评论': 10,
 '大会': 10,
 '独家': 1

In [98]:
inter_words = [k for k, v in cnt.items() if v > 3 and k not in label_names]
print(len(inter_words))
inter_words

5478


['中国',
 '时间',
 '北京',
 '市场',
 '问题',
 '记者',
 '方面',
 '经济',
 '大家',
 '企业',
 '美国',
 '国家',
 '行业',
 '中心',
 '内容',
 '情况',
 '手机',
 '全国',
 '媒体',
 '技术',
 '部分',
 '全球',
 '信息',
 '任务',
 '历史',
 '朋友',
 '专业',
 '领域',
 '英国',
 '城市',
 '上海',
 '电影',
 '感觉',
 '项目',
 '故事',
 '主题',
 '集团',
 '原因',
 '基本',
 '政府',
 '背景',
 '专家',
 '大学',
 '地方',
 '学生',
 '旗下',
 '网站',
 '论坛',
 '重点',
 '团队',
 '人士',
 '精神',
 '规定',
 '资源',
 '单位',
 '东西',
 '新闻',
 '经历',
 '阶段',
 '商业',
 '地点',
 '事业',
 '深圳',
 '数字',
 '工程',
 '高端',
 '交流',
 '规划',
 '区域',
 '品质',
 '主持人',
 '话题',
 '热情',
 '办法',
 '联系',
 '意识',
 '评价',
 '规范',
 '顶级',
 '电视',
 '影响力',
 '文章',
 '总裁',
 '亚洲',
 '总体',
 '双方',
 '公众',
 '观点',
 '评论',
 '大会',
 '独家',
 '层面',
 '人文',
 '眼球',
 '节目',
 '纪录',
 '分类',
 '形态',
 '外国',
 '公司',
 '世界',
 '消息',
 '国际',
 '方式',
 '产品',
 '基金',
 '系统',
 '品牌',
 '时候',
 '体验',
 '文化',
 '风格',
 '机会',
 '环境',
 '能力',
 '模式',
 '代表',
 '角色',
 '过程',
 '平台',
 '人们',
 '机构',
 '传统',
 '价值',
 '功能',
 '效果',
 '整体',
 '基础',
 '网络',
 '全面',
 '特色',
 '大量',
 '数据',
 '计划',
 '价格',
 '利用',
 '时代',
 '精彩',
 '个人',
 '网友',
 '全部',
 '空间',
 '现场